In [ ]:
#imports and installations that we need

!pip install transformers
!pip install -r https://raw.githubusercontent.com/huggingface/transformers/main/examples/research_projects/rag/requirements.txt
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.1/582.1 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#This is the webscraping code that takes in all the information from the wikipedia article including the tables,
# This is the webscraping code without scraping information from the links in the wiki page, because if I use dfs
# Google colab’s ram is being used up, so I don’t know if that works or not or maybe there’s a better way, we can mention the
# depth, I will try it out, But this code is without that part as I don’t know if it works or not.

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os



def parse_table(table):
    rows = table.find_all('tr')
    header = [th.text.strip() for th in rows[0].find_all('th')]

    # Skip tables without headers
    if not header:
        return None

    data = []
    max_columns = 0
    for row in rows[1:]:
        rowData = [td.text.strip() for td in row.find_all('td')]
        data.append(rowData)
        max_columns = max(max_columns, len(rowData))

    # Add default column names for any missing columns
    if len(header) < max_columns:
        for i in range(len(header), max_columns):
            header.append(f"Column_{i + 1}")

    return pd.DataFrame(data, columns=header)



# Set the URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/India"

# Send the request and get the content
response = requests.get(url)
if response.status_code == 200:

  soup = BeautifulSoup(response.text, 'html.parser')

  # Extract the main content container


  # Find all section headings
  section_headings = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

  # Create a directory to store the text files
  os.makedirs("wikipedia_sections", exist_ok=True)

  # Extract the page content and organize it under section headings
  for heading in section_headings:
      content = f"{heading.text}\n"
      for tag in heading.find_next_siblings():
          if tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
              break
          content += f"{tag.text}\n"

      # Save the content to a text file with the section heading as the filename
      filename = os.path.join("wikipedia_sections", f"{heading.text}.txt")
      with open(filename, "w", encoding='utf-8') as file:
          file.write(content)

  # Find all tables
  main_content = soup.find('div', {"id": "bodyContent"})
  tables = main_content.find_all('table')
  print(tables)

  # Scrape all tables and save them as DataFrames
  data_frames = []
  for table in tables:
      df = parse_table(table)
      if df is not None:
          data_frames.append(df)

  # Save tables to separate text files with the section heading as the filename
  table_count = 1
  for df in data_frames:
      filename = os.path.join("wikipedia_sections", f"Table_{table_count}.txt")
      with open(filename, "w", encoding='utf-8') as file:
          file.write(df.to_string(index=False))
      table_count += 1


[<table class="infobox ib-country vcard"><tbody><tr><th class="infobox-above adr" colspan="2"><div class="fn org country-name">Republic of India</div><div class="ib-country-names"><span title="ISO 15919 Indic (Hindi language) transliteration"><i lang="hi-Latn">Bhārat Gaṇarājya</i></span><br/><span style="font-size:85%;">(see <a href="/wiki/Names_of_India_in_its_official_languages" title="Names of India in its official languages">other regional names</a>)</span></div></th></tr><tr><td class="infobox-image" colspan="2"><div class="noresize" style="display:table; width:100%;">
<div style="display:table-cell; vertical-align:middle; padding-left:5px;">
<div style="padding-bottom:3px;"><span class="mw-image-border" typeof="mw:File"><a class="mw-file-description" href="/wiki/File:Flag_of_India.svg" title="Flag of India"><img alt="Horizontal tricolour flag bearing, from top to bottom, deep saffron, white, and green horizontal bands. In the centre of the white band is a navy-blue wheel with 24 

In [ ]:
## Now this code is for splitting these various articles into different passages, we need to do the same with both
## the text article and the tables, we need to just replace the file paths to suit the tables text files as well

import os
import urllib

titles = []
articles = []

i = 0

for filename in os.listdir("/content/wikipedia_sections"):

  with open("/content/wikipedia_sections/" + filename, "rb") as f:

    title = urllib.parse.unquote(filename[:-4])
    print(title)

    title = title.replace('_', ' ')

    if len(title) == 0 or (len(title.strip()) == 0):
      continue

    titles.append(title)
    articles.append(f.read().decode('utf-8'))

  i += 1

  if (i % 500 ) == 0:
    print('Processed {:,}'.format(i))

print("Done\n")

print("There are {:,} articles".format(len(articles)))

passage_titles = []

passage = []

for i in range(len(titles)):

  title = titles[i]
  article = articles[i]


  if len(article) == 0:
    continue
  words = article.split()

  for i in range(0, len(words), 100):

    chunk_words = words[i: i + 100]

    chunk = "".join(chunk_words)

    chunk = chunk.strip()

    if len(chunk) == 0:

      continue
    passage_titles.append(title)
    passage.append(chunk)

chunked_part = {'title': passage_titles, 'text': passage}

print("After splitting {:,} passages".format(len(chunked_part['title'])))



Cuisine
Architecture
Table_23
Early modern India
Table_6
Table_16
Socio-economic challenges
Biodiversity
Table_5
Table_24
Table_13
Table_20
Literature
See also
Performing arts and media
Table_2
Table_4
Table_25
Table_18
States
Table_27
Energy
Culture
Table_19
Geography
External links
Clothing
Union territories
Table_1
Table_21
Table_10
Sports and recreation
Industries
India
Table_8
Ancient India
Table_17
Table_15
Government
Notes
Demographics, languages and religion
Administrative divisions
Table_26
Modern India
Table_22
Table_11
Table_7
History
Table_9
References
Foreign, economic and strategic relations
Politics
Economy
Visual art
Medieval India
Politics and government
Education
Society
Table_12
Bibliography
Table_14
Table_3
Contents
Etymology
Done

There are 64 articles
After splitting 414 passages


In [ ]:
## These code snippets consist of tokenising, setting up pytorch, creating an embeddings column convert the ##chunked_part
## or the articles splitting part into a dataframe using panda , and add then add the embeddings column and add the FAISS
## index to help with the querying

!pip install transformers
!pip install faiss-cpu

from transformers import DPRContextEncoderTokenizerFast
import faiss


ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")


num_passages = len(chunked_part['title'])

outputs = ctx_tokenizer(chunked_part['title'], chunked_part['text'], truncation=True, padding="longest", return_tensors="pt")

input_ids = outputs['input_ids']


!pip install torch
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("using a gpu")
else:
  device = torch.device("cpu")
  print("using a cpu")


from transformers import DPRContextEncoder

ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

ctx_encoder = ctx_encoder.to(device = device)

import torch
import math
torch.set_grad_enabled(False)

step = 0

batch_size = 41

num_passages = input_ids.size()[0]
print(num_passages)

num_batches = math.ceil(num_passages/batch_size)

embeds_batches = []

step = 0


for i in range(0, num_passages , batch_size):


  print("Step is: ", step)
  step += 1


  batch_ids = input_ids[i: i + 41, :]
  batch_ids = batch_ids.to(device)
  outputs = ctx_encoder(
      batch_ids,
      return_dict = True

  )
  embeddings = outputs['pooler_output']
  embeddings = embeddings.detach().cpu().numpy()
  embeds_batches.append(embeddings)

!pip install numpy

import numpy as np

embeddings = np.concatenate(embeds_batches, axis=0)
print(embeddings.shape)


!pip install pandas

from datasets import Dataset

import  pandas as pd

df = pd.DataFrame(chunked_part)



dataset = Dataset.from_pandas(df)

print(dataset)
print(embeddings.shape[0])
embs = []



for i in range(embeddings.shape[0]):

   embs.append(embeddings[i, :])

print(len(embs))

dataset = dataset.add_column("embeddings", embs)

print(dataset)


# dim = 512
# m = 128




# index = faiss.IndexHNSWFlat(dim , m , faiss.METRIC_INNER_PRODUCT)
# # index.add(embeddings)
# # index.train(embeddings)
# dataset.add_faiss_index(column="embeddings", index_name="embeddings", custom_index = index, faiss_verbose=True)





The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


using a cpu


Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


414
Step is:  0
Step is:  1
Step is:  2
Step is:  3
Step is:  4
Step is:  5
Step is:  6
Step is:  7
Step is:  8
Step is:  9
Step is:  10
(414, 768)
Dataset({
    features: ['title', 'text'],
    num_rows: 414
})
414
414
Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 414
})


In [ ]:

import faiss



dim = 768
m = 128




index = faiss.IndexHNSWFlat(dim , m , faiss.METRIC_INNER_PRODUCT)

dataset = dataset.add_faiss_index(column="embeddings", index_name="embeddings", custom_index = index, faiss_verbose=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
## Now all is left is to get the retriever, the tokeniser and Whether RAGSequenceForGeneration or RAGTokenForGeneration
## In order to get the question, tokenise it and then get the answer


from transformers import RagRetriever

retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", use_dummy_dataset=False, indexed_dataset=dataset,index_name="embeddings")

from transformers import RagTokenizer, RagSequenceForGeneration

tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")

model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq", retriever = retriever )
# /* Your question based on the database */
question="What is the national animal of India?"

input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

generated = model.generate(input_ids)

generator = tokenizer.batch_decode(generated, skip_special_tokens = True)[0]

print("The question is: " + question)
print("The answer is : "+ generator)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

The question is: What is the national animal of India?
The answer is :  chimpanzees
